In [1]:
!pip install transformers
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install bitsandbytes accelerate
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 20.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import re

In [ ]:
df1 = pd.read_csv('/content/content_simulation_test_company.xlsx_0-5000.csv')
df2 = pd.read_csv('/content/content_simulation_test_company.xlsx_5000-10000.csv')
df3 = pd.read_excel('/content/content_simulation_test_company.xlsx')

result = pd.concat([df1, df2], ignore_index=True)
print(len(result))

In [ ]:
result.columns = ['Image Captions']
result.head()

In [ ]:
colNames = list(df3.columns) + ['image captions']

In [ ]:
result = pd.concat([result, df3], axis=1)

In [ ]:
result["Tweet Text Generated"] = ""

In [ ]:
result.head()

In [ ]:
for i in range(10000):
  conversation='You are the tweet predictor bot for company all over the world. You have vast knowledge of how the tweet are designed. You should predict the tweet depend on the likes it got, image caption given by the user and utilize your knowledge to know about the company at that time  if you can. You should predict the tweet such that it was written by a human. Tweet should have maximum of 3 sentences.'
  conversation = conversation + '\n' + f'''Generate a tweet for a social media post based on the following information:
  Likes: {result.loc[i]['likes']}
  Image Caption: {result.loc[i]['Image Captions']}
  Date and Time: {result.loc[i]['date']}
  Inferred Company: {result.loc[i]['inferred company']}
  Username : {result.loc[i]['username']}
  Craft a compelling and engaging tweet that incorporates the given image caption ideas, and try to include ideas if get from the inferred company name as well as username. Ensure the tweet is suitable for a social media platform and captures the audience's attention. You don't need to provide any explanation.'''

  sequences = pipeline(
      conversation,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      return_full_text=False,
      eos_token_id=tokenizer.eos_token_id,
      max_length=1000,
  )
  for seq in sequences:
      result.at[i,"Tweet Text Generated"] = result.loc[i]['Tweet Text Generated'] + seq['generated_text']
  if i%25 == 0:
    print(i)
  print(result.loc[i]['Tweet Text Generated'])